In [ ]:
from openai import OpenAI
import pandas as pd
import openpyxl
import pyarrow as pa
import pyarrow.parquet as pq

In [ ]:
file = pd.ExcelFile('data_for_db_corrections.xlsx')  
df_excel = pd.read_excel(file, "Sheet1")

In [ ]:
df_excel.info()

In [ ]:
answers = df_excel['Ответ из БЗ'].tolist()
display(answers[:5])

In [ ]:
prompts = [x + '\ОТВЕТ: {answer}' for x in ['У тебя есть ответ. Придумай вопрос, для которого этот ответ правильный и адекватный.',
'У тебя есть ответ. Придумай вопрос, для которого этот ответ правильный и адекватный, который мог бы задать подросток.',
'У тебя есть ответ. Придумай вопрос, для которого этот ответ правильный и адекватный, который мог бы задать сильно занятой человек.',
'У тебя есть ответ. Придумай вопрос, для которого этот ответ правильный и адекватный, который мог бы задать начинающий блогер.',
'У тебя есть ответ. Придумай вопрос, для которого этот ответ правильный и адекватный, который могла бы задать домохозяйка.']]

In [ ]:
client = OpenAI(api_key = '')
def complete(query: str, instruction: str):
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        temperature=1,
        messages=[
            {"role": "system", "content": instruction},
            {"role": "user", "content": query}
        ]
    )
    return completion.choices[0].message.content

In [ ]:
synth_questions = []

In [ ]:
for answer in answers:
    new_questions = []
    new_questions.append(answer)
    for prompt in prompts:
        new_questions.append(complete(answer, prompt))
    synth_questions.append(new_questions)

In [ ]:
synth_questions

In [ ]:
custom_columns = ['Ответ из БЗ', 'Синт. вопрос 1', 'Синт. вопрос 2', 'Синт. вопрос 3', 'Синт. вопрос 4', 'Синт. вопрос 5']
df = pd.DataFrame(synth_questions, columns = custom_columns)

In [ ]:
df.info()

In [ ]:
df_excel = df_excel.drop(df_excel.columns[[0]], axis=1)
df = df.drop(df.columns[[0]], axis=1)

In [ ]:
synth_q_from_a = df_excel.join(df, how='left', sort=False)

In [ ]:
synth_q_from_a.describe()

In [ ]:
table = pa.Table.from_pandas(synth_q_from_a)
pq.write_table(table, 'corrections_with_synth_questions_from_answer' + '.parquet')